In [ ]:
# ========================================================
# 🔹 CHyPS v3.1.3: Classificação Híbrida com k-Fold=5 + Filtro 3 Classes + Gráfico Completo
# ========================================================
# Autor: Abraão Gualberto Nazário (versão atualizada por Gemini)
# Data: Junho/2025
# --------------------------------------------------------

# --- SETUP DO AMBIENTE ---
try:
    import google.colab
    IN_COLAB = True
    print("🔹 Ambiente: Google Colab detectado.")
    !pip install -q pandas openpyxl scikit-learn spacy openai matplotlib
except ImportError:
    IN_COLAB = False
    print("🔹 Ambiente local detectado.")

# Imports
import pandas as pd
import numpy as np
import spacy
import matplotlib.pyplot as plt
import json
import time
import os
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from openai import OpenAI

# --- Setup spaCy ---
try:
    nlp = spacy.load('pt_core_news_lg')
    print("✅ Modelo spaCy 'pt_core_news_lg' carregado.")
except OSError:
    print("⚠️ Modelo 'pt_core_news_lg' não encontrado. Tentando 'pt_core_news_md'...")
    try:
        nlp = spacy.load('pt_core_news_md')
    except OSError:
        print("📥 Baixando modelo 'pt_core_news_md'...")
        spacy.cli.download('pt_core_news_md')
        nlp = spacy.load('pt_core_news_md')
    print("✅ Modelo spaCy 'pt_core_news_md' carregado.")

# --- Setup da API do LLM (Groq) ---
api_key = os.getenv('GROQ_API_KEY')
if not api_key:
    print("⚠️ Variável de ambiente 'GROQ_API_KEY' não encontrada.")
    try:
        from google.colab import userdata
        api_key = userdata.get('GROQ_API_KEY')
        print("✅ Chave da API da Groq carregada do Colab Secrets.")
    except Exception:
        api_key = input("Insira sua chave de API da Groq manualmente: ")

llm_client = OpenAI(api_key=api_key, base_url="https://api.groq.com/openai/v1")
print("✅ Cliente da API Groq configurado.")

# ========================================================
# 🔹 FUNÇÕES DO ALGORITMO CHyPS
# ========================================================

def preprocess_with_spacy(text, nlp_model):
    doc = nlp_model(str(text).lower())
    lemmas = [token.lemma_ for token in doc if not token.is_stop and not token.is_punct and not token.is_space]
    return ' '.join(lemmas)

def llm_classify_real(text_to_classify, llm_client, model_name):
    system_prompt = """
    Você é um especialista em classificação. Sua tarefa é analisar o texto e classificá-lo em UMA das seguintes categorias: '15', '16' ou '4'.
    Sua resposta DEVE ser um objeto JSON: {"classe": "15", "confianca": 0.9}
    """
    user_prompt = f'Texto para classificar:\n---\n{text_to_classify}\n---\nSua resposta em formato JSON:'
    try:
        response = llm_client.chat.completions.create(
            model=model_name,
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ],
            temperature=0,
            response_format={"type": "json_object"}
        )
        json_response = json.loads(response.choices[0].message.content)
        return str(json_response.get('classe', 'NI')), float(json_response.get('confianca', 0.0))
    except Exception as e:
        print(f"❌ Erro na chamada da API LLM: {e}")
        return 'NI', 0.0

def chamar_llm_com_retry(text, tentativas=3, pausa=2):
    for tentativa in range(tentativas):
        try:
            classe, confianca = llm_classify_real(text, llm_client, "llama3-8b-8192")
            if classe != 'NI':
                return classe, confianca
        except Exception as e:
            print(f"Tentativa {tentativa+1}/{tentativas} falhou com erro: {e}")
        time.sleep(pausa)
    return 'NI', 0.0

def chyps_classify(texto_original, texto_limpo, rf_model, vectorizer, limiar_tau):
    texto_tfidf = vectorizer.transform([texto_limpo])
    rf_pred_proba = rf_model.predict_proba(texto_tfidf)
    confianca_rf = np.max(rf_pred_proba)
    predicao_rf = rf_model.classes_[np.argmax(rf_pred_proba)]
    predicao_llm, confianca_llm = chamar_llm_com_retry(texto_original)
    comprimento = len(texto_original.split())
    W_rf, W_llm = (0.3, 0.7) if comprimento > limiar_tau else (0.7, 0.3)
    score_rf = confianca_rf * W_rf
    score_llm = confianca_llm * W_llm
    return predicao_llm if score_llm > score_rf else predicao_rf

def plotar_resultados(resultados_df):
    resultados_df.plot(kind='bar', figsize=(14, 8))
    plt.title('Comparativo Detalhado de Métricas por Modelo', fontsize=18, weight='bold')
    plt.ylabel('Pontuação', fontsize=14)
    plt.xlabel('Modelo', fontsize=14)
    plt.ylim(0, 1.05)
    plt.xticks(rotation=0, ha='center', fontsize=12)
    plt.legend(title='Métricas', fontsize=11, loc='upper left')
    plt.tight_layout()
    plt.savefig('comparacao_detalhada_metricas.png', dpi=300)
    plt.show()

# ========================================================
# 🔹 EXECUÇÃO PRINCIPAL (K-Fold=5)
# ========================================================

def main():
    # --- Parâmetros ---
    NOME_DO_ARQUIVO = 'dataset_aumentado.csv'
    K_FOLDS = 5
    PAUSA_API = 0.5
    LIMIAR_TAU = 20
    AVERAGE_TYPE = 'weighted'

    print(f"🔹 Carregando dados de: {NOME_DO_ARQUIVO}")
    try:
        df = pd.read_csv(NOME_DO_ARQUIVO)
    except FileNotFoundError:
        print(f"❌ ERRO: Arquivo '{NOME_DO_ARQUIVO}' não encontrado.")
        return
    df['categoria'] = df['categoria'].astype(str)

    CATEGORIAS_VALIDAS = ['15', '16', '4']
    df = df[df['categoria'].isin(CATEGORIAS_VALIDAS)].copy()
    print("🔹 Filtro aplicado: categorias válidas =", CATEGORIAS_VALIDAS)
    print(df['categoria'].value_counts())

    print("🔹 Pre-processando textos com spaCy...")
    df['texto_limpo'] = df['texto'].apply(lambda x: preprocess_with_spacy(x, nlp))

    X_original = df['texto']
    X_limpo = df['texto_limpo']
    y = df['categoria']

    skf = StratifiedKFold(n_splits=K_FOLDS, shuffle=True, random_state=42)

    metricas_rf, metricas_svm, metricas_chyps = [], [], []

    fold = 1
    for train_idx, test_idx in skf.split(X_limpo, y):
        print(f"\n🔹 Fold {fold}/{K_FOLDS}")
        fold += 1

        X_train_limpo, X_test_limpo = X_limpo.iloc[train_idx], X_limpo.iloc[test_idx]
        X_train_original, X_test_original = X_original.iloc[train_idx], X_original.iloc[test_idx]
        y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

        vectorizer = TfidfVectorizer(max_features=5000, ngram_range=(1, 2))
        X_train_tfidf = vectorizer.fit_transform(X_train_limpo)
        X_test_tfidf = vectorizer.transform(X_test_limpo)

        rf_model = RandomForestClassifier(n_estimators=100, random_state=42, class_weight='balanced')
        rf_model.fit(X_train_tfidf, y_train)

        svm_model = SVC(kernel='linear', probability=True, random_state=42, class_weight='balanced')
        svm_model.fit(X_train_tfidf, y_train)

        y_pred_rf = rf_model.predict(X_test_tfidf)
        y_pred_svm = svm_model.predict(X_test_tfidf)

        y_pred_chyps = []
        for texto_orig, texto_limpo in zip(X_test_original, X_test_limpo):
            pred = chyps_classify(texto_orig, texto_limpo, rf_model, vectorizer, LIMIAR_TAU)
            y_pred_chyps.append(pred)
            time.sleep(PAUSA_API)

        for metricas, y_pred in zip(
            [metricas_rf, metricas_svm, metricas_chyps],
            [y_pred_rf, y_pred_svm, y_pred_chyps]
        ):
            metricas.append({
                'Acurácia': accuracy_score(y_test, y_pred),
                'Precisão': precision_score(y_test, y_pred, average=AVERAGE_TYPE, zero_division=0),
                'Recall': recall_score(y_test, y_pred, average=AVERAGE_TYPE, zero_division=0),
                'F1-Score': f1_score(y_test, y_pred, average=AVERAGE_TYPE, zero_division=0)
            })

    resultados_df = pd.DataFrame({
        'Modelo': ['Random Forest', 'SVM', 'CHyPS'],
        'Acurácia': [np.mean([m['Acurácia'] for m in metricas_rf]), np.mean([m['Acurácia'] for m in metricas_svm]), np.mean([m['Acurácia'] for m in metricas_chyps])],
        'Precisão': [np.mean([m['Precisão'] for m in metricas_rf]), np.mean([m['Precisão'] for m in metricas_svm]), np.mean([m['Precisão'] for m in metricas_chyps])],
        'Recall': [np.mean([m['Recall'] for m in metricas_rf]), np.mean([m['Recall'] for m in metricas_svm]), np.mean([m['Recall'] for m in metricas_chyps])],
        'F1-Score': [np.mean([m['F1-Score'] for m in metricas_rf]), np.mean([m['F1-Score'] for m in metricas_svm]), np.mean([m['F1-Score'] for m in metricas_chyps])]
    }).set_index('Modelo')

    print("\n📊 RESULTADOS FINAIS (Média 5 folds)")
    print(resultados_df)

    plotar_resultados(resultados_df)
    resultados_df.to_csv('resultados_chyps_kfold_completo.csv')

    print("\n✅ Execução completa com gráfico completo (v3.1.3). 🚀")

if __name__ == '__main__':
    main()


🔹 Ambiente: Google Colab detectado.
⚠️ Modelo 'pt_core_news_lg' não encontrado. Tentando 'pt_core_news_md'...
✅ Modelo spaCy 'pt_core_news_md' carregado.
⚠️ Variável de ambiente 'GROQ_API_KEY' não encontrada.
✅ Chave da API da Groq carregada do Colab Secrets.
✅ Cliente da API Groq configurado.
🔹 Carregando dados de: dataset_aumentado.csv
🔹 Filtro aplicado: categorias válidas = ['15', '16', '4']
categoria
15    402
4      31
16     22
Name: count, dtype: int64
🔹 Pre-processando textos com spaCy...

🔹 Fold 1/5

🔹 Fold 2/5


In [ ]:
# Carregar o dataset manualmente só para analisar as classes
df = pd.read_csv('dataset_aumentado.csv')
df['categoria'] = df['categoria'].astype(str)

# Ver quantos exemplos tem por classe
classe_counts = df['categoria'].value_counts()
print("Contagem de exemplos por classe:")
print(classe_counts)


In [ ]:
# Checar quantos exemplos tem por classe
classe_counts = df['categoria'].value_counts()
print("Contagem de exemplos por classe:")
print(classe_counts)